In [1]:
from __future__ import division
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing import Imputer
#from analytics_tools3 import freq_discrete
import pandas as pd
import numpy as np
import graphviz
import seaborn as sns
pd.set_option('display.max_columns',500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline

In [2]:
df = pd.read_csv('C:/Users/cody8/Desktop/Documentos/Escuela/AMV/parcial 3 y 4/entrenar2.csv')

In [3]:
df.head()

,Cuadrante,c_Total de crimenes,c_HOMICIDIO_DOLOSO,c_LESIONES_POR_ARMA_DE_FUEGO,c_ROBO_A_BORDO_DE_METRO_CV,c_ROBO_A_BORDO_DE_MICROBUS_CV,c_ROBO_A_BORDO_DE_MICROBUS_SV,c_ROBO_A_BORDO_DE_TAXI_CV,c_ROBO_A_CASA_HABITACION_CV,c_ROBO_A_CUENTAHABIENTE_CV,c_ROBO_A_NEGOCIO_CV,c_ROBO_A_REPARTIDOR_CV,c_ROBO_A_REPARTIDOR_SV,c_ROBO_A_TRANSEUNTE_CV,c_ROBO_A_TRANSEUNTE_SV,c_ROBO_A_TRANSPORTISTA_CV,c_ROBO_A_TRANSPORTISTA_SV,c_ROBO_DE_VEHICULO_AUTOMOTOR_CV,c_ROBO_DE_VEHICULO_AUTOMOTOR_SV,c_VIOLACION,obj
0,1,205,6,1,0,19,0,1,4,1,24,4,1,29,2,1,0,48,62,2,0
1,2,41,1,0,0,1,1,1,1,0,13,0,1,2,0,0,0,9,10,1,0
2,3,140,1,3,0,3,0,1,4,4,15,9,3,34,0,0,0,17,46,0,0
3,4,107,10,5,0,2,0,0,4,0,4,20,3,32,1,0,0,10,12,4,0
4,5,109,2,1,0,3,0,2,1,1,19,6,1,16,1,3,0,21,30,2,0


In [4]:
var_disc = [x for x in df.columns if x.startswith('d_')]
var_cont = [x for x in df.columns if x.startswith('c_')]
# obj = d_ROBO_A_BORDO_DE_METRO_SV Jul Ag Sept
X = df[var_cont+var_disc].copy()
y = df['obj'].copy()

print len(var_cont)
print len(var_disc)

19
0


In [5]:
#im = Imputer(strategy='mean')
#im.fit(X,y)
#X = im.transform(X)

In [26]:
Xt,Xv,yt,yv = train_test_split(X,y,train_size=0.7)

print Xt.shape
print Xv.shape
print yt.shape
print yv.shape

(415, 19)
(178, 19)
(415L,)
(178L,)


In [27]:
%%time
param_grid=dict(max_depth=range(2,8),
            max_features=range(2,X.shape[1]),
            max_leaf_nodes=range(5,21),
            min_samples_leaf=[0.05])
dt=DecisionTreeClassifier()
grid = GridSearchCV(dt,param_grid,
                            cv=10,
                            scoring='roc_auc',
                            n_jobs=-1)
grid.fit(Xt,yt)

print grid.best_params_
print grid.best_estimator_
print grid.best_score_

{'max_features': 16, 'max_leaf_nodes': 15, 'max_depth': 2, 'min_samples_leaf': 0.05}
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=16, max_leaf_nodes=15, min_impurity_split=1e-07,
            min_samples_leaf=0.05, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.888474851788
Wall time: 3min 9s


In [8]:
%%time
param_grid=dict(max_depth=range(2,8),
            max_features=range(2,X.shape[1]),
            max_leaf_nodes=range(5,21),
            min_samples_leaf=[0.05])
dt=DecisionTreeClassifier()
rsgrid = RandomizedSearchCV(dt,
                            param_grid,
                            cv=10,
                            scoring='roc_auc',
                            n_jobs=-1,
                            n_iter=50)
rsgrid.fit(Xt,yt)

print rsgrid.best_params_
print rsgrid.best_estimator_
print rsgrid.best_score_

{'max_features': 15, 'max_leaf_nodes': 13, 'max_depth': 4, 'min_samples_leaf': 0.05}
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=15, max_leaf_nodes=13, min_impurity_split=1e-07,
            min_samples_leaf=0.05, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
0.881425702811
Wall time: 30 s


In [28]:
dt = grid.best_estimator_
#dt = rsgrid.best_estimator_

In [29]:
dt.fit(Xt,yt)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=16, max_leaf_nodes=15, min_impurity_split=1e-07,
            min_samples_leaf=0.05, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [30]:
print accuracy_score(y_pred=dt.predict(Xv),y_true=yv)
print accuracy_score(y_pred=dt.predict(Xt),y_true=yt)

0.88202247191
0.89156626506


In [31]:
print roc_auc_score(y_score=dt.predict_proba(Xt)[:,1],y_true=yt)
print roc_auc_score(y_score=dt.predict_proba(Xv)[:,1],y_true=yv)

0.912826128357
0.872140522876


In [32]:
with open('arbolex.dot','wb') as f:
    tree.export_graphviz(class_names=['good','bad'],
                         decision_tree=dt,
                         feature_names=var_cont+var_disc,
                         out_file=f)
    f.close()

In [33]:
predicarb=pd.Series(dt.predict_proba(Xv)[:,1])

In [34]:
predicarb

0     0.055
1     0.055
2     0.000
3     0.792
4     0.055
5     0.000
6     0.022
7     0.792
8     0.022
9     0.055
10    0.022
11    0.000
12    0.022
13    0.055
14    0.055
15    0.792
16    0.500
17    0.190
18    0.022
19    0.000
20    0.000
21    0.000
22    0.000
23    0.055
24    0.055
25    0.055
26    0.055
27    0.190
28    0.022
29    0.000
       ... 
148   0.000
149   0.000
150   0.022
151   0.055
152   0.055
153   0.055
154   0.055
155   0.055
156   0.425
157   0.055
158   0.055
159   0.055
160   0.055
161   0.792
162   0.055
163   0.425
164   0.000
165   0.000
166   0.500
167   0.425
168   0.190
169   0.792
170   0.000
171   0.500
172   0.000
173   0.022
174   0.425
175   0.190
176   0.055
177   0.000
dtype: float64

In [35]:
vf = pd.read_csv('C:/Users/cody8/Desktop/Documentos/Escuela/AMV/parcial 3 y 4/validar2.csv')

In [36]:
tf = pd.read_csv('C:/Users/cody8/Desktop/Documentos/Escuela/AMV/parcial 3 y 4/entrenar2.csv')

In [37]:
var_disc = [x for x in tf.columns if x.startswith('d_')]
var_cont = [x for x in tf.columns if x.startswith('c_')]
## obj = d_ROBO_A_BORDO_DE_METRO_SV Jul Ag Sept
Xw = tf[var_cont+var_disc].copy()
yw = tf['obj'].copy()



var_discf = [x for x in vf.columns if x.startswith('d_')]
var_contf = [x for x in vf.columns if x.startswith('c_')]
## obj = d_ROBO_A_BORDO_DE_METRO_SV Jul Ag Sept
Xv = vf[var_contf+var_discf].copy()



In [38]:
#Comprobamos que la dimensión de nuestras bases sea correcta
print Xw.shape
print Xv.shape
print yw.shape

(593, 19)
(199, 19)
(593L,)


In [39]:
dt.fit(Xw,yw)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=16, max_leaf_nodes=15, min_impurity_split=1e-07,
            min_samples_leaf=0.05, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [40]:
print roc_auc_score(y_score=dt.predict_proba(Xw)[:,1],y_true=yw)

0.903150537634


In [41]:
print accuracy_score(y_pred=dt.predict(Xw),y_true=yw)

0.897133220911


In [42]:
print X.shape
print Xv.shape
print y.shape

(593, 19)
(199, 19)
(593L,)


In [43]:
predicssj2=pd.Series(dt.predict_proba(Xv)[:,1])

In [47]:
predicssj2#.to_csv('s_311093416_p2.csv')

0     0.057
1     0.057
2     0.153
3     0.842
4     0.057
5     0.057
6     0.153
7     0.842
8     0.057
9     0.000
10    0.057
11    0.057
12    0.057
13    0.057
14    0.000
15    0.568
16    0.568
17    0.057
18    0.000
19    0.568
20    0.000
21    0.000
22    0.000
23    0.057
24    0.057
25    0.000
26    0.057
27    0.057
28    0.057
29    0.568
       ... 
169   0.000
170   0.000
171   0.057
172   0.000
173   0.000
174   0.000
175   0.000
176   0.000
177   0.000
178   0.326
179   0.057
180   0.057
181   0.326
182   0.000
183   0.153
184   0.057
185   0.057
186   0.057
187   0.153
188   0.000
189   0.000
190   0.057
191   0.057
192   0.842
193   0.568
194   0.000
195   0.057
196   0.568
197   0.842
198   0.000
dtype: float64

In [77]:
# Nombre del árbol gen arbolex.dot